In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
from pathlib import Path
import gc
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
plt.figure(figsize=(12, 12))

logger = logger_func()

2019-09-28 21:39:48,551 func.utils 347 [INFO]    [logger_func] start 


<Figure size 864x864 with 0 Axes>

In [2]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMN_GROUP = 'DT-M'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, COLUMN_GROUP, 'is_train', 'datetime', 'date', 'year', 'month', 'DT-M']

def filter_feature(path):
    if path.count(''):
        return True
    else:
        return False

paths_train = glob('../submit/re_sub/*_train.gz')
paths_test  = glob('../submit/re_sub/*_test.gz')
paths_train += glob('../submit/add_feature/*_train.gz')
paths_test  += glob('../submit/add_feature/*_test.gz')
print(len(paths_train))

paths_train_feature = []
paths_test_feature  = []

# df_train = reduce_mem_usage( parallel_load_data(paths_train) )
# df_test  = reduce_mem_usage( parallel_load_data(paths_test) )
df_train = parallel_load_data(paths_train)
df_test  = parallel_load_data(paths_test)

679


Process ForkPoolWorker-94:
Process ForkPoolWorker-54:
Process ForkPoolWorker-1:
Process ForkPoolWorker-34:
Process ForkPoolWorker-51:
Process ForkPoolWorker-53:


In [3]:
# data = pd.concat([df_train, df_test], axis=0)
startdate = datetime.datetime(2017,12,1)

# data['datetime'] = data['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
# data['year'] = data['datetime'].map(lambda x: x.year)
# data['month'] = data['datetime'].map(lambda x: x.month)
# data['month'] = data['month'].map(lambda x: 5 if x==6 else x)

# data['DT-M'] = data[['year', 'month']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)


df_train['datetime'] = df_train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
df_train['year'] = df_train['datetime'].map(lambda x: x.year)
df_train['month'] = df_train['datetime'].map(lambda x: x.month)
df_train['month'] = df_train['month'].map(lambda x: 5 if x==6 else x)
df_train['DT-M'] = df_train[['year', 'month']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)

df_test['datetime'] = df_test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
df_test['year'] = df_test['datetime'].map(lambda x: x.year)
df_test['month'] = df_test['datetime'].map(lambda x: x.month)
df_test['month'] = df_test['month'].map(lambda x: 5 if x==6 else x)

df_test['DT-M'] = df_test[['year', 'month']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)

In [74]:
#========================================================================
# Bear's ID
#========================================================================
df_user_id_bear = pd.read_csv('../output/same_user_pattern/20190901_user_ids_share.csv')
# data = pd.concat([df_train, df_test], axis=0)
# col_bear = 'user_id_bear'
# data.columns = [col.replace('ker__', '') for col in data.columns]

# data.set_index(COLUMN_ID, inplace=True)
# df_user_id_bear = pd.read_csv('../output/same_user_pattern/20190901_user_ids_share.csv').set_index(COLUMN_ID)
# data[col_bear] = df_user_id_bear['predicted_user_id']

# max_id = data[col_bear].max()

# data.loc[data[data[col_bear].isnull()].index, col_bear] = np.arange(data[data[col_bear].isnull()].shape[0]) + max_id+1
# data.reset_index(inplace=True)
# data[col_bear] = data[col_bear].astype('int')
# data[col_bear].head()

In [62]:
#========================================================================
# Bear's score
#========================================================================

from sklearn.metrics import roc_auc_score
bear = pd.read_csv('../input/20190913_ieee__bear_probing.csv').iloc[:, [0, 1, 2, 3, 4, 6]]
# bear = bear[bear[COLUMN_TARGET]==-1]
bear = bear.iloc[:, [0,1,2,3,5]]
bear.columns = [COLUMN_ID, COLUMN_DT, col_bear, 'data_type', 'bear_probing']
bear = bear.merge(data[[COLUMN_ID, 'DT-M']], how='inner', on=COLUMN_ID)
submission = pd.read_csv('../input/sample_submission.csv')
submission.columns = [COLUMN_ID, 'pred']


def bear_validation(test_pred):
    submission['pred'] = test_pred
    bear_score = submission.merge(bear, how='inner', on=COLUMN_ID)
    public  = bear_score[bear_score['data_type']=='test_public']
    private = bear_score[bear_score['data_type']=='test_private']
    
    public_score = roc_auc_score(public[COLUMN_TARGET].values, public['pred'].values)
    private_score = roc_auc_score(private[COLUMN_TARGET].values, private['pred'].values)
    all_score = roc_auc_score(bear_score[COLUMN_TARGET].values, bear_score['pred'].values)

    return public_score, private_score, all_score

In [34]:
#========================================================================
# Join Prediction
#========================================================================
# pred = read_pkl_gzip('../output/pred_result/20190925_1450__CV0-9581588018233685__all_preds.gz').iloc[:, 1].values
# data['pred_holy'] = pred

# oof_haku = pd.read_csv('../output/oof/oof.csv').iloc[:, 1].values
# test_haku = pd.read_csv('../output/oof/submission.csv').iloc[:, 1].values
# data['pred_haku'] = np.hstack((oof_haku, test_haku))

# oof_hmd = pd.read_csv('../output/oof/20190925_hmdhmd_oof.csv').iloc[:, 1].values
# test_hmd = pd.read_csv('../output/oof/20190925_hmdhmd_pred.csv').iloc[:, 1].values
# data['pred_hmd'] = np.hstack((oof_hmd, test_hmd))

# oof_bear = pd.read_csv('../output/oof/20190925_all_uid_agg_stats_lr001_oof_features1381_oof0.95_pub0.984_pri0.989.csv')
# test_bear = pd.read_csv('../output/oof/20190925_all_uid_agg_stats_lr001_pred_features1381_oof0.95_pub0.984_pri0.989.csv')
# df_bear = pd.concat([oof_bear, test_bear], axis=0)
# df_bear.columns = [COLUMN_ID, 'pred_bear']
# data = data.merge(df_bear, how='inner', on=COLUMN_ID)


pred = read_pkl_gzip('../output/pred_result/20190925_1450__CV0-9581588018233685__all_preds.gz').iloc[:len(df_train), 1].values
df_train['pred_holy'] = pred
oof_haku = pd.read_csv('../output/oof/oof.csv').iloc[:, 1].values
df_train['pred_haku'] = oof_haku

oof_hmd = pd.read_csv('../output/oof/20190925_hmdhmd_oof.csv').iloc[:, 1].values
df_train['pred_hmd'] = oof_hmd

oof_bear = pd.read_csv('../output/oof/20190925_all_uid_agg_stats_lr001_oof_features1381_oof0.95_pub0.984_pri0.989.csv')
oof_bear.columns = [COLUMN_ID, 'pred_bear']
df_train = df_train.merge(oof_bear, how='inner', on=COLUMN_ID)


pred = read_pkl_gzip('../output/pred_result/20190925_1450__CV0-9581588018233685__all_preds.gz').iloc[len(df_train):, 1].values
df_test['pred_holy'] = pred

test_haku = pd.read_csv('../output/oof/submission.csv').iloc[:, 1].values
df_test['pred_haku'] = test_haku

test_hmd = pd.read_csv('../output/oof/20190925_hmdhmd_pred.csv').iloc[:, 1].values
df_test['pred_hmd'] = test_hmd

test_bear = pd.read_csv('../output/oof/20190925_all_uid_agg_stats_lr001_pred_features1381_oof0.95_pub0.984_pri0.989.csv')
test_bear.columns = [COLUMN_ID, 'pred_bear']
df_test = df_test.merge(test_bear, how='inner', on=COLUMN_ID)

In [36]:
#========================================================================
# 検証するbear's idでfilter
#========================================================================
data.sort_values(by=COLUMN_DT, inplace=True)
bear_first = data.groupby(col_bear)['DT-M'].first()
bear_cnt = data.groupby(col_bear)['DT-M'].count()

bear_test_user = bear_first[bear_first>='2018-7'].index
bear_multi_cnt_user = bear_cnt[bear_cnt>3].index
bear_valid_user = list(set(bear_test_user) & set(bear_multi_cnt_user))
print(len(bear_valid_user))

df_bear_valid = bear[bear[col_bear].isin(bear_valid_user)]
df_bear_valid = df_bear_valid.merge(data[[COLUMN_ID, 'pred_holy', 'pred_haku', 'pred_hmd', 'pred_bear']], how='inner', on=COLUMN_ID)

pd.set_option('max_rows', 1400)
df_bear_valid.sort_values(by=[col_bear, COLUMN_DT], ascending=False)

13180


In [9]:
cnt_adv = pd.concat([
df_train['528__ugr_R_emaildomain_C1_C14_ratio_agg_V35_52_mean_mean'].value_counts().rename('cnt_train'), 
df_test['528__ugr_R_emaildomain_C1_C14_ratio_agg_V35_52_mean_mean'].value_counts().rename('cnt_test')
], axis=1)
cnt_adv.sort_values(by='cnt_test', ascending=False, inplace=True)
cnt_adv

,cnt_train,cnt_test
0.539019,335751.0,274585.0
0.922614,38772.0,41394.0
0.590437,20809.0,19492.0
0.961942,18891.0,17591.0
0.868339,15277.0,14189.0
0.566206,16716.0,12656.0
0.569855,14096.0,11782.0
1.149457,8830.0,8467.0
0.814798,9239.0,7357.0
0.556883,8502.0,6677.0


In [41]:
from sklearn.metrics import roc_auc_score
df_train['round_ugr'] = df_train['528__ugr_R_emaildomain_C1_C14_ratio_agg_V35_52_mean_mean'].map(lambda x: np.round(x, 2))
list_val = []

for val in tqdm(sorted(df_train['round_ugr'].unique().tolist())):
    
    tmp = df_train[df_train['round_ugr']==val]
    
    if len(tmp)==0:
        continue
    
    y_train = tmp[COLUMN_TARGET].values
    haku = tmp['pred_haku'].values
    holy = tmp['pred_holy'].values
    hmd = tmp['pred_hmd'].values
    bear = tmp['pred_bear'].values
    cnt = tmp.shape[0]
    
    try:
        score_haku = roc_auc_score(y_train, haku)
        score_hmd = roc_auc_score(y_train, hmd)
        score_holy = roc_auc_score(y_train, holy)
        score_bear = roc_auc_score(y_train, bear)
        mean_haku = np.mean(haku)
        mean_hmd  = np.mean(hmd)
        mean_holy = np.mean(holy)
        mean_bear = np.mean(bear)
        max_val = np.max(y_train)
#         print(f" * ugr: {val} cnt: {cnt} haku: {score_haku} hmd: {score_hmd} holy: {score_holy} bear: {score_bear}")
        list_val.append([val, cnt, score_haku, score_hmd, score_holy, score_bear, max_val, mean_haku, mean_hmd, mean_holy, mean_bear])
    except ValueError:
        mean_haku = np.mean(haku)
        mean_hmd  = np.mean(hmd)
        mean_holy = np.mean(holy)
        mean_bear = np.mean(bear)
        max_val = np.max(y_train)
#         print(f" * ugr: {val} cnt: {tmp.shape[0]} y_train: {np.max(y_train)} ")
        list_val.append([val, cnt, np.nan, np.nan, np.nan, np.nan, max_val, mean_haku, mean_hmd, mean_holy, mean_bear])

100%|██████████| 272/272 [00:05<00:00, 51.71it/s] 


In [43]:
pd.set_option('max_rows', 300)
# ugr_train = pd.DataFrame(list_val, columns=['ugr', 'cnt', 'score_haku', 'score_hmd', 'score_holy', 'score_bear', 'max_target', 'mean_haku', 'mean_hmd', 'mean_holy', 'mean_bear'])
ugr_train.columns = [f"train_{col}" if not col.count('ugr') else col  for col in ugr_train.columns]

In [45]:
df_test['round_ugr'] = df_test['528__ugr_R_emaildomain_C1_C14_ratio_agg_V35_52_mean_mean'].map(lambda x: np.round(x, 2))
list_val = []

for val in tqdm(sorted(df_test['round_ugr'].unique().tolist())):
    
    tmp = df_test[df_test['round_ugr']==val]
    
    if len(tmp)==0:
        continue
    
    y_test = tmp[COLUMN_TARGET].values
    haku = tmp['pred_haku'].values
    holy = tmp['pred_holy'].values
    hmd = tmp['pred_hmd'].values
    bear = tmp['pred_bear'].values
    cnt = tmp.shape[0]
    
    try:
        mean_haku = np.mean(haku)
        mean_hmd  = np.mean(hmd)
        mean_holy = np.mean(holy)
        mean_bear = np.mean(bear)
        max_val = np.max(y_test)
        list_val.append([val, cnt, max_val, mean_haku, mean_hmd, mean_holy, mean_bear])
    except ValueError:
        mean_haku = np.mean(haku)
        mean_hmd  = np.mean(hmd)
        mean_holy = np.mean(holy)
        mean_bear = np.mean(bear)
        max_val = np.max(y_test)
        list_val.append([val, cnt, max_val, mean_haku, mean_hmd, mean_holy, mean_bear])

100%|██████████| 325/325 [00:04<00:00, 66.17it/s] 


In [46]:
ugr_test = pd.DataFrame(list_val, columns=['ugr', 'cnt', 'max_target', 'mean_haku', 'mean_hmd', 'mean_holy', 'mean_bear'])
ugr_test.columns = [f"test_{col}" if not col.count('ugr') else col  for col in ugr_test.columns]
ugr_test.head()

,ugr,test_cnt,test_max_target,test_mean_haku,test_mean_hmd,test_mean_holy,test_mean_bear
0,0.00,1,NaN,0.001627,0.003237,0.003757,0.006765
1,0.46,106,NaN,0.158311,0.152592,0.169974,0.187896
2,0.50,177,NaN,0.026778,0.025598,0.043991,0.033125
3,0.51,1512,NaN,0.008860,0.012417,0.012599,0.012087
4,0.52,4374,NaN,0.003852,0.006193,0.005105,0.005251


In [47]:
df_ugr = ugr_train.merge(ugr_test, how='outer', on='ugr')
df_ugr.head()

,ugr,train_cnt,train_score_haku,train_score_hmd,train_score_holy,train_score_bear,train_max_target,train_mean_haku,train_mean_hmd,train_mean_holy,train_mean_bear,test_cnt,test_max_target,test_mean_haku,test_mean_hmd,test_mean_holy,test_mean_bear
0,0.00,6.0,1.000000,1.000000,1.000000,1.000000,1.0,0.117848,0.047873,0.127307,0.116812,1.0,NaN,0.001627,0.003237,0.003757,0.006765
1,0.46,165.0,0.978148,0.973449,0.968515,0.982848,1.0,0.152751,0.171911,0.196523,0.178794,106.0,NaN,0.158311,0.152592,0.169974,0.187896
2,0.49,153.0,0.989237,0.935421,0.979452,0.979452,1.0,0.031673,0.033229,0.032788,0.031838,91.0,NaN,0.037527,0.035277,0.045750,0.041782
3,0.50,353.0,0.972805,0.993902,0.996829,0.987195,1.0,0.036091,0.036758,0.041254,0.037983,177.0,NaN,0.026778,0.025598,0.043991,0.033125
4,0.51,3229.0,0.976165,0.971749,0.962694,0.968476,1.0,0.007095,0.007907,0.006946,0.007189,1512.0,NaN,0.008860,0.012417,0.012599,0.012087


In [50]:
df_ugr.to_csv('../output/0928_ieee__528__ugr_R_emaildomain_C1_C14_ratio_agg_V35_52_mean_mean__round2__score.csv', index=False)

In [97]:
# col_bear = 'predicted_user_id'
# cols_pred = [col for col in df_train.columns if col.count('pred')]

# if col_bear not in df_train.columns:
#     df_train = df_train.merge(df_user_id_bear[[COLUMN_ID, col_bear]], how='left', on=COLUMN_ID)

tmp = df_train[(0.959<df_train['round_ugr']) 
         &
         (df_train['round_ugr']<0.961)][[COLUMN_ID, 'DT-M', 'datetime', COLUMN_TARGET] + cols_pred]
tmp.sort_values(by=COLUMN_TARGET, ascending=False, inplace=True)
tmp = tmp[tmp[COLUMN_TARGET]==1]

low_ids = tmp[col_bear].unique()

In [100]:
train_low = df_train[df_train[col_bear].isin(low_ids)][[COLUMN_ID, 'DT-M', 'datetime', COLUMN_TARGET] + cols_pred]

In [101]:
train_low.sort_values(by=[col_bear, 'datetime'], inplace=True)
train_low

,TransactionID,DT-M,datetime,isFraud,pred_holy,pred_haku,pred_hmd,pred_bear,predicted_user_id
98374,3085374,2017-12,2017-12-23 23:27:01,1,0.005418,0.006594,0.018037,0.001160,288.0
98565,3085565,2017-12,2017-12-23 23:53:14,1,0.004442,0.006098,0.010895,0.001512,288.0
572063,3559063,2018-5,2018-05-25 20:18:53,1,0.025139,0.001826,0.008054,0.010378,1033.0
396006,3383006,2018-3,2018-03-26 04:48:35,0,0.044509,0.017446,0.047083,0.419595,1035.0
396484,3383484,2018-3,2018-03-26 15:35:29,0,0.017173,0.048781,0.072576,0.218130,1035.0
397410,3384410,2018-3,2018-03-26 20:40:37,1,0.158475,0.193318,0.384720,0.583477,1035.0
441082,3428082,2018-4,2018-04-09 16:39:52,1,0.036392,0.059748,0.399370,0.147729,1035.0
441096,3428096,2018-4,2018-04-09 16:46:43,1,0.048099,0.067002,0.385572,0.144854,1035.0
443417,3430417,2018-4,2018-04-10 14:41:15,1,0.057148,0.041687,0.436016,0.058694,1035.0
470564,3457564,2018-4,2018-04-20 15:36:31,1,0.038984,0.031150,0.324351,0.079661,1035.0
